# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62013

In [4]:
client_tiface = await new_transport_interface("0.0.0.0", 0)
client_tiface.run()

server_tiface = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface.run()

14:15:25.828563 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
14:15:25.830897 [INFO ] - [T] Attempting to bind to 0.0.0.0:0
14:15:25.831016 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
14:15:25.836375 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
14:15:25.850537 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7f07da85f530>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

# Filter layer setup

In [5]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

{'c': <FilterInterface at 0x7f07da768d30>,
 's': <FilterInterface at 0x7f07da768930>}

Find methods to run

In [6]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command', 'command_response', 'get_notifications', 'response', 'run']

Run Client and Server Filter!

In [7]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

## Filter layer operation for Pings

### As a client, add server (both our local and Chococat) as ping endpoints

In [8]:
import socket
choco_ip = socket.gethostbyname_ex('chococat.conwayste.rs')[-1][0]
choco_ip

'157.230.134.224'

In [9]:
server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")
choco_ept  = EndpointW(f"{choco_ip}:2016")

filter_cmd = FilterCmdW("addpingendpoints", endpoints=[server_ept, choco_ept])
filter_cmd

AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)] }

In [10]:
await client_fiface.command_response(filter_cmd)

14:15:26.363968 [TRACE] - [F<-A,C] New command: AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)] }


Accepted

The client will repeatedly send pings. Need `LATENCY_FILTER_DEPTH` pings (currently 4) to be received back from server before we know the latency.

In [11]:
time.sleep(3.0)
server_notifications = server_fiface.get_notifications()
server_notifications

14:15:26.505187 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:26.505296 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 1211382832661210982 } }] }
14:15:26.505357 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 18154187388394093579 } }] }
14:15:26.505399 [TRACE] - [F<-T,R] Command Accepted
14:15:26.505406 [TRACE] - [F<-T,R] Command Accepted
14:15:26.505446 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 18154187388394093579 } }
14:15:26.505479 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:40370), took packet GetStatus { ping: PingPong { non

14:15:27.137642 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 7759249542875136059 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:27.137674 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 7759249542875136059 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:27.137686 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(14)
14:15:27.305667 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:27.305720 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 6 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 13609237666484760437 } }] }
14:15:27.305775 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint

14:15:27.926825 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 9147211536663251283 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:27.926888 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 9147211536663251283 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:27.926903 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(11)
14:15:28.105361 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:28.105406 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 7 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 12516783725390697757 } }] }
14:15:28.105471 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint

14:15:28.730113 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 5196655197530605987 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:28.730149 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 5196655197530605987 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:28.730180 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(13)
14:15:28.905099 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:28.905137 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 13 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 18315424828529832622 } }] }
14:15:28.905205 [TRACE] - [T<-F,C] Processing command SendPackets { endpoin

[]

In [12]:
client_notifications = client_fiface.get_notifications()
client_notifications

[PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: None, server_name: "placeholder server name", server_version: "placeholder server version", room_count: 12456, player_count: 1234 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: None, server_name: "placeholder server name", server_version: "placeholder server version", room_count: 12456, player_count: 1234 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: None, server_name: "placeholder server name", server_version: "placeholder server version", room_count: 12456, player_count: 1234 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Offici

Sleeping again....

In [13]:
time.sleep(10.0)
client_notifications = client_fiface.get_notifications()
client_notifications

14:15:29.505656 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:29.505695 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 17 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 1919335318360167651 } }] }
14:15:29.505750 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 18 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 12281198811794437917 } }] }
14:15:29.505780 [TRACE] - [F<-T,R] Command Accepted
14:15:29.505784 [TRACE] - [F<-T,R] Command Accepted
14:15:29.505798 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 12281198811794437917 } }
14:15:29.505823 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:40370), took packet GetStatus { ping: PingPong { n

14:15:30.305942 [TRACE] - [F<-T,R] Command Accepted
14:15:30.305949 [TRACE] - [F<-T,R] Command Accepted
14:15:30.306128 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 6207926461857261780 } }
14:15:30.306150 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:40370), took packet GetStatus { ping: PingPong { nonce: 6207926461857261780 } }
14:15:30.306162 [INFO ] - [F] Sending Status packet PingPong { nonce: 6207926461857261780 } back to client Endpoint(127.0.0.1:40370)
14:15:30.306174 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:40370), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 21 }, retry_interval: 0ns }], packets: [Status { pong: PingPong { nonce: 6207926461857261780 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" }] }
14:15:30.306211 [TRACE] - [F<-T,R] Command Accepted
14:15:30.306232 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 620792

14:15:31.105606 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:31.105634 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 17 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 16730132465439133919 } }] }
14:15:31.105696 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 18 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 15608873106967446473 } }] }
14:15:31.105734 [TRACE] - [F<-T,R] Command Accepted
14:15:31.105740 [TRACE] - [F<-T,R] Command Accepted
14:15:31.105777 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 15608873106967446473 } }
14:15:31.105802 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:40370), took packet GetStatus { ping: PingPong { 

14:15:31.928825 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 2368341528137770075 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:31.928863 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 2368341528137770075 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:31.928875 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(11)
14:15:32.105417 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:32.105461 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 25 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 10536907197717461782 } }] }
14:15:32.105517 [TRACE] - [T<-F,C] Processing command SendPackets { endpoin

14:15:32.727278 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 3857847214781243976 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:32.727341 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 3857847214781243976 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:32.727359 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(11)
14:15:32.904822 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:32.904859 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 39 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 15876959106991289311 } }] }
14:15:32.904923 [TRACE] - [T<-F,C] Processing command SendPackets { endpoin

...

14:15:36.905559 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:36.905602 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 55 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 17320653362766261382 } }] }
14:15:36.905663 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 56 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 9750322443583405892 } }] }
14:15:36.905700 [TRACE] - [F<-T,R] Command Accepted
14:15:36.905707 [TRACE] - [F<-T,R] Command Accepted
14:15:36.905732 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 9750322443583405892 } }
14:15:36.905746 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:40370), took packet GetStatus { ping: PingPong { no

14:15:37.705543 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:37.705586 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 61 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 2165034813444079358 } }] }
14:15:37.705643 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 62 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 16271757539919580124 } }] }
14:15:37.705673 [TRACE] - [F<-T,R] Command Accepted
14:15:37.705679 [TRACE] - [F<-T,R] Command Accepted
14:15:37.705684 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 16271757539919580124 } }
14:15:37.705701 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:40370), took packet GetStatus { ping: PingPong { n

14:15:38.527327 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 16479734022031772689 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:38.527361 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 16479734022031772689 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:38.527373 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(11)
14:15:38.705441 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:38.705482 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 68 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 13664610384825801436 } }] }
14:15:38.705544 [TRACE] - [T<-F,C] Processing command SendPackets { endpo

14:15:39.328600 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 5785381278979069601 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:39.328707 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 5785381278979069601 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
14:15:39.328744 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(11)


[PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: Some(0), server_name: "placeholder server name", server_version: "placeholder server version", room_count: 12456, player_count: 1234 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: Some(11), server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: Some(0), server_name: "placeholder server name", server_version: "placeholder server version", room_count: 12456, player_count: 1234 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: Some(11), server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: Some(0), server_name: "placeholder server name", server_version: "placeholder server version", room_count: 12456, player_count: 1234 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: Some(11),

In [14]:
await client_fiface.command_response(FilterCmdW("clearpingendpoints"))

14:15:39.439279 [TRACE] - [F<-A,C] New command: ClearPingEndpoints
14:15:39.439350 [INFO ] - [F<-A,C] clearing ping endpoints: [Endpoint(157.230.134.224:2016), Endpoint(127.0.0.1:62013)]
14:15:39.439467 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(157.230.134.224:2016) }
14:15:39.439487 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(127.0.0.1:62013) }
14:15:39.439511 [TRACE] - [F<-T,R] Command Accepted
14:15:39.439516 [TRACE] - [F<-T,R] Command Accepted


Accepted

In [15]:
client_notifications = client_fiface.get_notifications()
client_notifications

[]

### Now shutdown

In [16]:
time.sleep(1)
await client_fiface.command_response(FilterCmdW("shutdown", graceful=True))

14:15:40.484796 [TRACE] - [F<-A,C] New command: Shutdown { graceful: true }
14:15:40.484817 [INFO ] - [F] shutting down
14:15:44.312417 [INFO ] - [F<-T,N] Endpoint(127.0.0.1:40370) timed-out. Dropping.
14:15:44.312440 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(127.0.0.1:40370) }
14:15:44.312454 [TRACE] - [F<-T,R] Command Accepted
